In [13]:
pip install dash==1.8.0

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install tensorflow==2.1.0

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install keras==2.3.1

Note: you may need to restart the kernel to use updated packages.


In [16]:
import base64
import os
from urllib.parse import quote as urlquote
from flask import Flask, send_from_directory
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
from pathlib import Path
from dash.dependencies import Input, Output, State
from PIL import Image
import numpy as np
import glob
from skimage import transform
import keras.preprocessing.image as image
from tensorflow.keras.models import load_model
import cv2

In [18]:
print(keras.__version__)

NameError: name 'keras' is not defined

In [19]:
image_directory = 'aptos2019-blindness-detection/test_images'
list_of_images = [ os.path.basename(x) for x in glob.glob(( os.path.join(image_directory, '*.png') ))]
static_image_route = 'data:image/png;base64,'

In [33]:
app = dash.Dash(
    __name__, 
    #external_stylesheets=external_stylesheets,
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}]
    )
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}


app.layout = html.Div(
    [
        
    html.Div(
        [
        html.Div(
            [
                html.Div(
                    [
                        html.H1(
                            "Blindness Detection using Deep Learning",
                            className="uppercase title",
                        ),
                        html.Span(
                            "Image classifier using dense neural nets", style={'padding': '10px'}
                        ),
                        html.Br(),                        
                        html.Span(
                            " The following are the five classes base on severity of blindness", style={'padding': '10px'}
                        ),
                        html.Br(),                        
                        html.Span(
                            " ", style={'padding': '10px'}
                        ),  
                        html.Br(),                     
                        html.Span("1. No Diabetic retinopathy   ", style={'color': '#4ce80e'} ),
                        html.Span("2. Mild   ",      style={'color': '#83db1f'} ),
                        html.Span("3. Moderate   ",       style={'color': '#ebcd0e'} ),
                        html.Span("4. Severe   ",      style={'color': '#db3a2c'} ),
                        html.Span("5. Proliferative Diabetic retinopathy   ", style={'color': '#e01b1b'} ),
                        html.Br(),  
                    ], style = {'background':'#ffffff'}
                ),
            ],
        className="app__header",
    ),
        
            html.Div( children = [
                dcc.Dropdown(
                    id='image-dropdown',
                    options=[{'label': i, 'value': i} for i in list_of_images],
                    value=  list_of_images[0]
                ),
            html.Div([dcc.Store(id='intermediate-value')], style={'display': 'none'}),
            html.Div(
                [   html.H3("Original Image"),
                    html.Img(id='image',style={'display': 'inline-block','height':'20%', 'width':'20%'})]
                , className="six columns"),
            html.Div(
                [   html.H3("Preprocessed Image"),
                    html.Img(id='image_2',style={'display': 'inline-block','height':'20%', 'width':'20%'})]
                , className="six columns"),
            html.Div(
                [ html.H3("Predicted Class"),
                      html.P(id="predicted-class")]
            )
            ]    
            , className="row"),   
        ], className="app__container",
    )
    ],style = {'background':'#ffffff'}
)

In [30]:

def load(filename):
    image_path = image_directory + '/' + filename
    np_image = Image.open(image_path)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (512, 512, 3))
    np_image = image.img_to_array(np_image)
    np_image = np.expand_dims(np_image, axis=0)
    predict = model.predict(np_image)
    result_class = ["No Diabetic retinopathy","Mild","Moderate","Severe","Proliferative Diabetic retinopathy"]
    result1 = result_class[np.argmax(predict,axis=1)[0]]
    return result1



In [31]:
def preprocess_image(filename, sigmaX = 10):
    image_path = image_directory + '/' + filename
    image_temp = cv2.imread(image_path)
    image_temp = cv2.cvtColor(image_temp, cv2.COLOR_BGR2RGB)
    #image_temp = cv2.resize(image_temp, (224,224))
    image_temp = cv2.addWeighted(image_temp, 4, cv2.GaussianBlur(image_temp, (0,0), sigmaX), -4, 128)
    image = Image.fromarray(image_temp, 'RGB')
    #im_b64 = base64.b64encode(image).decode("utf-8")
    return image


In [32]:
model = load_model('model_resnet50_final.h5')

In [34]:
@app.callback(
    [Output('image', 'src'),
     Output('image_2', 'src'),
     Output('predicted-class','children')],
    [Input('image-dropdown', 'value')])
def update_image_src(value):
    image_path = image_directory + '/' + value
    encoded_image = base64.b64encode(open(image_path, 'rb').read())
    preprocessed_image = preprocess_image(value)
    result = load(value)
    #result_class = predict_class(np_image)
    return ('data:image/png;base64,{}'.format(encoded_image.decode()),
            preprocessed_image,result)




In [35]:
if __name__ == "__main__":
     app.run_server(
        port=8080,
        host='0.0.0.0',
    )

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Aug/2020 18:56:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2020 18:56:54] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2020 18:56:54] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2020 18:56:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2020 18:57:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2020 18:57:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2020 18:57:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2020 18:57:19] "POST /_dash-update-component HTTP/1.1" 200 -
